In [739]:
import random
import jieba.analyse
import pandas as pd
import numpy as np

## 1. 加载停用词

In [740]:
# stop_words_path = "C:\\Users\\Name\\PycharmProjects\\NLP_project\\resources\\stopwords-master\\百度停用词表.txt"
# stop_words_path = "C:\\Users\\Name\\PycharmProjects\\NLP_project\\resources\\stopwords-master\\哈工大停用词表.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_projects/resources/stopwords-master/哈工大停用词表.txt"
stop_words_path = "/home/yang/PycharmProjects/NLP_projects/resources/stopwords-master/中文停用词表.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_projects/resources/stopwords-master/modified_stopwords.txt"

# stop_words_path = "/home/yang/PycharmProjects/NLP_project/resources/stopwords-master/中文停用词表.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_project/resources/stopwords-master/百度停用词表.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_project/resources/stopwords-master/modified_stopwords.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_project/resources/stopwords-master/四川大学机器智能实验室停用词库.txt"

stop_words = pd.read_csv(stop_words_path,index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stop_words = stop_words['stopword'].values

## 2. 加载语料

In [741]:
# data_path = "C:\\Users\\Name\\PycharmProjects\\NLP_project\\resources\\面试-训练测试集.xlsx"
data_path = "/home/yang/PycharmProjects/NLP_projects/resources/面试-训练测试集.xlsx"
predict_data_path = "/home/yang/PycharmProjects/NLP_projects/resources/面试-待预测集.xlsx"
# data_path = "/home/yang/PycharmProjects/NLP_project/resources/面试-训练测试集.xlsx"
data = pd.read_excel(data_path)
predict_data = pd.read_excel(predict_data_path)


## 3.定义文本预处理函数

* 去无关字符，去停用词，分词
* 参数 titles, contents, sentences, labels

In [742]:
def is_float(s):
    s = str(s)
    if s.count('.') ==1:
        left = s.split('.')[0]
        right = s.split('.')[1]
        if right.isdigit():
            if left.count('-')==1 and left.startswith('-'):
                num = left.split['-'][-1]
                if num.isdigit():
                    return True
            elif left.isdigit():
                return True
    return False

def is_percents(s):
    s = str(s)
    if s.count('.') ==1:
        left = s.split('.')[0]
        right = s.split('.')[1]
        right = right.split('%')[0]
        if right.isdigit():
            if left.count('-')==1 and left.startswith('-'):
                num = left.split['-'][-1]
                if num.isdigit():
                    return True
            elif left.isdigit():
                return True
            
    elif s.count(".") != 1:
        left = s.split('%')[0]
        if left.isdigit():
            return True
               
    return False

def is_all_chinese(strs):
    for _char in strs:
        if not '\u4e00' <= _char <= '\u9fa5':
            return False
    return True


def preprocess_text(title, content, sentences, label):
    
#     处理标题
    title_info = jieba.lcut(title)

    title_info = [v for v in title_info if not str(v).isdigit()]#去数字
    title_info = list(filter(lambda x:x.strip(), title_info))   #去左右空格
    

    title_info = list(filter(lambda x:len(x)>1, title_info)) #长度为1的字符
    title_info = list(filter(lambda x:x not in stop_words, title_info)) #去掉停用词
            
#     处理文章内容        
    segs=jieba.lcut(content)
    segs = [v for v in segs if not str(v).isdigit()]#去数字
    segs = [v for v in segs if not is_float(v)] #去小数
    segs = [v for v in segs if not is_percents(v)] #去百分数

    segs = list(filter(lambda x:x.strip(), segs))   #去左右空格
    segs = list(filter(lambda x:len(x)>1, segs)) #长度为1的字符
    segs = list(filter(lambda x:x not in stop_words, segs)) #去掉停用词
    
    
#     print("内容信息是:", segs)
    segs = title_info + segs  
    segs = [v for v in segs if is_all_chinese(str(v))] #去字母词
    
    sentences.append((" ".join(segs), label))# 打标签

## 4. 文本预处理

In [743]:
sentences = []

for index, row in data.iterrows():
#     print(row['title'], row['content'])
    title = row['title']
    content = row['content']
    label = row['label']
    preprocess_text(title, content, sentences, label)
    

In [744]:


pos_sentences = []
neg_sentences = []
neu_sentences = []

for seg in sentences:
    if seg[1] == "正面":
        pos_sentences.append(seg)
    elif seg[1] == "负面":
        neg_sentences.append(seg)
    elif seg[1] == "中性":
        neu_sentences.append(seg)
        

## 5. 抽取词向量特征

* 测试了两种提取方法，CountVectorizer词频提取和TfidfVectorizer的td-idf特性提取
* CountVectorizer提取词频特征后，在训练集上有明显过拟合现象
* Tf-idf模型能更有效避免噪声词的干扰，在测试集上表现更好，这里用Tf-idf来构建词袋模型

In [745]:
# 1. 抽取特征， 定义词袋模型

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# vec = CountVectorizer(
#     analyzer='word', # tokenise by character ngrams
#     ngram_range=(1,2),  # use ngrams of size 1 and 2
#     max_features=1000,  # keep the most common 1000 ngrams
# )

vec = TfidfVectorizer(
    analyzer='word', # tokenise by character ngrams
    ngram_range=(1,2),  # use ngrams of size 1 and 2
    max_features= None,  # keep the most common 1000 ngrams
    use_idf=1,smooth_idf=1,sublinear_tf=1
)



In [746]:
# 2. 语料切分, 分出测试集用于验证
# 因为负面，中性的数据占比过少，需手动调整在训练集中的比例，防止这两类训练过少
from sklearn.model_selection import train_test_split
pos_x, pos_y = zip(*pos_sentences)
neg_x, neg_y = zip(*neg_sentences)
neu_x, neu_y = zip(*neu_sentences)
x_train_pos, x_test_pos, y_train_pos, y_test_pos = train_test_split(pos_x, pos_y,test_size=0.48, random_state=1)
x_train_neg, x_test_neg, y_train_neg, y_test_neg = train_test_split(neg_x, neg_y,test_size=0.08, random_state=2)
x_train_neu, x_test_neu, y_train_neu, y_test_neu = train_test_split(neu_x, neu_y,test_size=0.1, random_state=3)

x_train = x_train_pos + x_train_neg + x_train_neu
x_test = x_test_pos + x_test_neg + x_test_neu
y_train = y_train_pos + y_train_neg + y_train_neu
y_test = y_test_pos + y_test_neg + y_test_neu

train_data = []
test_data = []

for i in range(len(x_train)):
    train_data_pair = [x_train[i],y_train[i]]
    train_data.append(train_data_pair)
    
for i in range(len(x_test)):
    test_data_pair = [x_test[i],y_test[i]]
    test_data.append(test_data_pair)

# 打乱数据顺序
random.shuffle(train_data)
random.shuffle(test_data)

x_train, y_train = zip(*train_data)
x_test, y_test = zip(*test_data)


187
112


## 6. 定义模型

In [747]:
# 3. 把训练数据转为词袋模型
vec.fit(x_train)
x_train_data = vec.transform(x_train)

In [748]:
# 4.算法建模与模型训练 （贝叶斯）
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(x_train_data, y_train)
# 5. 评估结果 AUC值
print("多项贝叶斯模型评价，训练集正确率:",classifier.score(vec.transform(x_train), y_train))


多项贝叶斯模型评价，训练集正确率: 0.89171974522293


In [749]:
# 4.算法建模与模型训练 （SVM）

from sklearn.svm import SVC
# svm = SVC(kernel='linear')
svm = SVC(C=1.0,kernel='linear',gamma='auto')
svm.fit(x_train_data, y_train)

print("SVM模型评价，训练集正确率",svm.score(x_train_data, y_train))


SVM模型评价，训练集正确率 0.964968152866242


In [750]:
import sklearn.neural_network as sk_nn
model = sk_nn.MLPClassifier(activation='tanh',solver='adam',alpha=0.0001,learning_rate='adaptive',learning_rate_init=0.01,max_iter=4000)
model.fit(x_train_data,y_train)

print("神经网络模型评价，训练集正确率", model.score(x_train_data, y_train))

acc=model.score(vec.transform(x_test),y_test) #根据给定数据与标签返回正确率的均值


神经网络模型评价，训练集正确率 0.9904458598726115


## 7.检验模型

In [751]:
print("多项贝叶斯模型评价，测试集正确率:",classifier.score(vec.transform(x_test), y_test))
print("SVM模型评价，测试集正确率",svm.score(vec.transform(x_test), y_test))
print('神经网络模型评价, 测试集正确率:',acc)

多项贝叶斯模型评价，测试集正确率: 0.9081081081081082
SVM模型评价，测试集正确率 0.8972972972972973
神经网络模型评价, 测试集正确率: 0.8648648648648649


### 目前准确率最高的模型在测试集上为90%左右, 保存所有训练的模型

In [752]:
from sklearn.externals import joblib

# 保存神经网络模型
joblib.dump(model,'mlp.model') 
# 加载神经网络模型
mlp_model=joblib.load('mlp.model')

# 保存多项贝叶斯分类器的模型
joblib.dump(classifier,'bayes.model')
# 加载贝叶斯模型
bayes_model = joblib.load('bayes.model')

# 保存神经网络模型
joblib.dump(svm,'svm.model') 
# 加载神经网络模型
svm_model=joblib.load('svm.model')

## 8. 预测分类

In [755]:
# 处理预测数据
predict_data_sentences = []

for index, row in predict_data.iterrows():
#     print(row['title'], row['content'])
    title = row['title']
    content = row['content']
    preprocess_text(title, content, predict_data_sentences, label= None)

x, none_labels = zip(*predict_data_sentences)
# vec.fit(x)
x = vec.transform(x)

# 应用模型进行预测
result=mlp_model.predict(x)
# result = bayes_model.predict(x)
# result = svm_model.predict(x)
print(len(result))

1999


In [757]:
from pyecharts.faker import Faker
from pyecharts import options as opts
from pyecharts.charts import Pie

pos = 0
neg = 0
neu = 0

for i in result:
    if i == '正面':
        pos = pos+ 1
    elif i == '负面':
        neg += 1
    elif i == '中性':
        neu += 1
    else:
        print('youyouyou')
print(pos)        
res = [pos, neg, neu] 
print(res)
attr = ['正面','负面','中性']


def pie_base() -> Pie:
    c = (
        Pie()
        .add("", [list(z) for z in zip(attr, res)])
        .set_global_opts(title_opts=opts.TitleOpts(title="类别分析"))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    )
    return c


pie= pie_base()
pie.render_notebook()


1796
[1796, 200, 3]


## 9. 将预测的标签写入到csv文件中

In [763]:
import xlsxwriter as xlsw

df_res = pd.DataFrame(result)
print(df_res)
writer = pd.ExcelWriter(predict_data_path, engine='xlsxwriter')
df_res.to_excel(writer,startrow=2, startcol=6, header=False, index=False)

       0
0     正面
1     正面
2     正面
3     正面
4     正面
5     正面
6     正面
7     正面
8     负面
9     正面
10    负面
11    负面
12    正面
13    正面
14    正面
15    正面
16    正面
17    正面
18    正面
19    正面
20    正面
21    正面
22    正面
23    正面
24    正面
25    正面
26    正面
27    正面
28    正面
29    正面
...   ..
1969  正面
1970  负面
1971  正面
1972  正面
1973  正面
1974  正面
1975  正面
1976  正面
1977  正面
1978  负面
1979  正面
1980  正面
1981  正面
1982  正面
1983  正面
1984  正面
1985  正面
1986  正面
1987  正面
1988  正面
1989  正面
1990  正面
1991  正面
1992  正面
1993  正面
1994  正面
1995  正面
1996  正面
1997  正面
1998  正面

[1999 rows x 1 columns]
